In [3]:
import praw
from services.database import get_session

reddit = praw.Reddit('textmining') # Siehe praw.ini im Hauptverzeichnis

In [17]:
from datetime import datetime

# Mach das zu 1 == 1 um neue Posts und Comments zu scrapen
if 0 == 1:
    subreddit = reddit.subreddit('news+politics')

    for submission in subreddit.hot(limit=100):

        submission.selftext = submission.selftext if submission.selftext else "null"
        insert_post_query = f"""
            INSERT INTO post (id, subreddit, selftext, url, title, created_utc)
            VALUES (
                '{submission.id}',
                '{submission.subreddit.display_name}',
                $${submission.selftext}$$,
                $${submission.url}$$,
                $${submission.title}$$,
                '{datetime.fromtimestamp(submission.created_utc)}'
            );
        """
        try:
            with get_session() as session:
                session.query(insert_post_query)
                print(f"Inserted post '{submission.title}'")
        except Exception as error:
            print(f"Ignoring post {submission.id} because of {error}")
            continue

        comments = submission.comments

        # Remove praw.MoreComments instances
        comments.replace_more(limit=0)
        for comment in comments.list():
             # Parent id should be "null" if the parent of a comment is a post (e.g. toplevel comment)
            parent_id = "null" if comment.parent_id.startswith("t3_") else f"'{comment.parent().id}'"
            # Escape any $ with a \
            comment_body = comment.body.replace('$', r'\$')

            insert_comment_query = f'''
                INSERT INTO comment (id, body, post_id, subreddit, parent_id, created_utc)
                VALUES (
                    '{comment.id}', 
                    $${comment_body}$$,
                    '{comment.submission.id}',
                    '{comment.submission.subreddit.display_name}',
                    {parent_id},
                    '{datetime.fromtimestamp(comment.created_utc)}'
                );
            '''
            try:
                with get_session() as session:
                    session.query(insert_comment_query)
            except queries.IntegrityError as error:
                print(f'Ignoring comment {comment.id} because of: {error}')
                continue

    print(f"Finished inserting.")

# Ausgabe der Anzahl Posts und Comments in der Datenbank

In [20]:
with get_session() as session:
    results = session.query("SELECT COUNT(*) as posts FROM post UNION SELECT COUNT(*) as comments FROM comment")
    print(results.items())

[{'posts': 836}, {'posts': 135302}]
